In [1]:
%matplotlib qt

import neuroseries as nts

import numpy as np
import matplotlib.pyplot as plt
import scipy.stats
import scipy.ndimage
import os

import scipy.signal
import scipy.stats

import cv2
import datetime

import bk.load 
import bk.plot
import bk.compute

import sklearn.manifold as skm
import scipy.stats
from matplotlib import cm

In [2]:
def pourcent_freezing(interval, freezing_intervals): 
    freezing_intervals_in_interval = freezing_intervals.intersect(interval)
    total_time_freezing = np.sum(freezing_intervals_in_interval.end-freezing_intervals_in_interval.start)
    total_time = interval.end-interval.start
    pourcentage = (total_time_freezing/total_time)*100
    return pourcentage

def all_pourcentages(intervals, freezing_intervals):
    pourcentages = []
    for i in range(len(intervals)):
        interval = intervals.loc[i]
        pourcentages.append(pourcent_freezing(interval, freezing_intervals))
    return pourcentages

def GetStartExp(Exp,Fs = 20000):
    
    if isinstance(Exp[0],(np.bool_,bool)):
        Exp = list(map(int,Exp))
    
    diff_Exp = np.diff(Exp)
    
    t_start = np.where(diff_Exp == 1)[0]    
    return t_start/Fs

Analysis for classical fear conditionning 

In [58]:
context_sess = ['Y:\classical_fear_conditionning\Rat09-20201118\Rat09_201118_180813',
                'Y:\classical_fear_conditionning\Rat10-20201118\Rat10_201118_182234', 
                'Y:\classical_fear_conditionning\Rat11-20201117\Rat11_201117_172705',
                'Y:\classical_fear_conditionning\Rat12-20201117\Rat12_201117_171302', 
                'Y:\classical_fear_conditionning\Rat01-20200630\Rat01_200630_183149',
                'Y:\classical_fear_conditionning\Rat02-20200707\Rat02_200707_172929']
fear_sess = ['Y:\classical_fear_conditionning\Rat09-20201118\Rat09_201118_110003',
             'Y:\classical_fear_conditionning\Rat10-20201118\Rat10_201118_111531',
             'Y:\classical_fear_conditionning\Rat11-20201117\Rat11_201117_112130',
             'Y:\classical_fear_conditionning\Rat12-20201117\Rat12_201117_110146',
            'Y:\classical_fear_conditionning\Rat01-20200630\Rat01_200630_114732',
            'Y:\classical_fear_conditionning\Rat02-20200707\Rat02_200707_114611']

tone_sess = ['Y:\classical_fear_conditionning\Rat09-20201118\Rat09_201118_170529', 
        'Y:\classical_fear_conditionning\Rat10-20201118\Rat10_201118_172028', 
        'Y:\classical_fear_conditionning\Rat11-20201117\Rat11_201117_183012',
        'Y:\classical_fear_conditionning\Rat12-20201117\Rat12_201117_181452', 
        'Y:\classical_fear_conditionning\Rat01-20200630\Rat01_200630_171900', 
        'Y:\classical_fear_conditionning\Rat02-20200707\Rat02_200707_183427' ]


intervals = np.linspace(0,570,20)
starts = intervals[0:-1]
ends = intervals[1:]
intervals = nts.IntervalSet(starts,ends,time_units='s')

digitalin_tone = [3,3,3,3,2,2]
digitalin_shock = [2,2,2,2,1,1]

In [49]:
t_start_exp_tone = [49.9391, 12.74455, 30.36235, 29.62595, 58.95, 17.8116]

ptotal = []
plt.figure()

for fichier,t_s,d_t in zip(tone_sess,t_start_exp_tone,digitalin_tone):
    os.chdir(fichier)
    pos = bk.load.DLC_pos()
    pos.index = pos.index
    tone = bk.compute.tone_intervals(bk.load.digitalin('digitalin.dat')[d_t,:])- t_s * 1_000_000
    speeds = bk.compute.speed(pos,2, columns_to_drop=['e_tail'])
    freeze = bk.compute.freezing_intervals(speeds, 25, mode='multiple_speed', clean=True) - t_s *1_000_000
    pourcents = all_pourcentages(intervals, freeze)
    ptotal.append(pourcents)
#     bk.plot.intervals(freeze,col = 'b')
    plt.plot(np.mean([starts,ends],0), pourcents, '-o',alpha = 0.5)
    
bk.plot.intervals(tone,'orange', alpha=0.5)    
plt.plot(np.mean([starts,ends],0),np.mean(ptotal,0),'*-r')
plt.legend(['Rat09', 'Rat10', 'Rat11', 'Rat12', 'Rat01', 'Rat02','Mean','Tone'])

Z:\Python\Billel\bk\compute.py:160: RuntimeWarning: overflow encountered in square
  v = np.sqrt(x_speed**2 + y_speed**2)


In [60]:
t_start_exp_fear = [37.7209,29.46495,34.7118,25.34385,45.9524,55.00735]

ptotal = []
plt.figure()

for fichier,t_s,d_s in zip(fear_sess,t_start_exp_fear, digitalin_shock):
    os.chdir(fichier)
    pos = bk.load.DLC_pos()
    pos.index = pos.index
    tone = bk.compute.tone_intervals(bk.load.digitalin('digitalin.dat')[d_s+1,:])- t_s * 1_000_000
    shock = bk.compute.tone_intervals(bk.load.digitalin('digitalin.dat')[d_s,:])- t_s * 1_000_000
    speeds = bk.compute.speed(pos,2, columns_to_drop=['e_tail'])
    freeze = bk.compute.freezing_intervals(speeds, 25, mode='multiple_speed', clean=True) - t_s *1_000_000
    pourcents = all_pourcentages(intervals, freeze)
    ptotal.append(pourcents)
#     bk.plot.intervals(freeze,col = 'b')
    plt.plot(np.mean([starts,ends],0), pourcents, '-o',alpha = 0.5)
bk.plot.intervals(shock,'green',alpha = 0.5)    
bk.plot.intervals(tone,'orange', alpha=0.5)    
plt.plot(np.mean([starts,ends],0),np.mean(ptotal,0),'*-r')
plt.legend(['Rat09', 'Rat10', 'Rat11', 'Rat12', 'Rat01', 'Rat02','Mean','Shock','Tone'])

In [62]:
t_start_exp_context = [34,25,36,26,38,58]

ptotal = []
plt.figure()

for fichier,t_s in zip(context_sess,t_start_exp_context):
    os.chdir(fichier)
    pos = bk.load.DLC_pos()
    pos.index = pos.index
    speeds = bk.compute.speed(pos,2, columns_to_drop=['e_tail'])
    freeze = bk.compute.freezing_intervals(speeds, 25, mode='multiple_speed', clean=True) - t_s *1_000_000
    pourcents = all_pourcentages(intervals, freeze)
    ptotal.append(pourcents)
#     bk.plot.intervals(freeze,col = 'b')
    plt.plot(np.mean([starts,ends],0), pourcents, '-o',alpha = 0.5)
plt.plot(np.mean([starts,ends],0),np.mean(ptotal,0),'*-r')
plt.legend(['Rat09', 'Rat10', 'Rat11', 'Rat12', 'Rat01', 'Rat02','Mean'])

In [61]:
for fichier in context_sess:
    os.chdir(fichier)
    pos = bk.load.DLC_pos()
    print(pos.as_units('s').index[-1]-575)
#     f_tone = GetStartExp(bk.load.digitalin('digitalin.dat')[1,:])
#     print(f_tone[f_tone>0])
    
    


34.458774999999946
[  3.0225    3.1475    3.2725  203.08695 353.30415 503.51535]
25.68372499999998
[  3.0225    3.1475    3.2725  203.08695 353.30415 503.51535]
36.234050000000025
[  3.0225    3.1475    3.2725  203.08695 353.30415 503.51535]
26.03037500000005
[  3.0225    3.1475    3.2725  203.08695 353.30415 503.51535]
38.39945
[  3.0225    3.1475    3.2725  203.08695 353.30415 503.51535]
58.11689999999999
[  3.0225    3.1475    3.2725  203.08695 353.30415 503.51535]


Analysis for PTSD fear conditionning 

In [22]:
neutral_sess = ['Y://fear_conditionning_ptsd\Rat13\Rat13-20201202\Rat13_201202_183404',
               'Y://fear_conditionning_ptsd\Rat14\Rat14-20201203\Rat14_201203_165203',
               'Y://fear_conditionning_ptsd\Rat15\Rat15-20201130\Rat15_201130_164002',
               'Y://fear_conditionning_ptsd\Rat16\Rat16-20201201\Rat16_201201_172742']



context_sess = ['Y://fear_conditionning_ptsd\Rat13\Rat13-20201203\Rat13_201203_154214',
               'Y://fear_conditionning_ptsd\Rat14\Rat14-20201204\Rat14_201204_172709',
               'Y://fear_conditionning_ptsd\Rat15\Rat15-20201201\Rat15_201201_155055',
               'Y://fear_conditionning_ptsd\Rat16\Rat16-20201202\Rat16_201202_171552']

fear_sess = ['Y://fear_conditionning_ptsd\Rat13\Rat13-20201203\Rat13_201203_112541',
            'Y://fear_conditionning_ptsd\Rat14\Rat14-20201204\Rat14_201204_113458',
            'Y://fear_conditionning_ptsd\Rat15\Rat15-20201201\Rat15_201201_115713',
            'Y://fear_conditionning_ptsd\Rat16\Rat16-20201202\Rat16_201202_132308']

tone_sess = ['Y://fear_conditionning_ptsd\Rat13\Rat13-20201203\Rat13_201203_164205',
            'Y://fear_conditionning_ptsd\Rat14\Rat14-20201204\Rat14_201204_161859',
            'Y://fear_conditionning_ptsd\Rat16\Rat16-20201202\Rat16_201202_182329']


intervals = np.linspace(0,240,13)
starts = intervals[0:-1]
ends = intervals[1:]
intervals = nts.IntervalSet(starts,ends,time_units='s')

digitalin_tone = [3,3,3,3]
digitalin_shock = [2,2,2,2]

In [15]:
for subsession in neutral_sess:
    os.chdir(subsession)
    f_tone = GetStartExp(bk.load.digitalin('digitalin.dat')[1,:])
    print(f_tone[f_tone>0])

[36.98515]
[25.79435]
[36.52745]
[34.03365]


In [24]:
t_start_exp_neutral = [36.98515,25.79435,36.52745,34.03365]

ptotal = []
plt.figure()

for fichier,t_s in zip(neutral_sess,t_start_exp_neutral):
    os.chdir(fichier)
    print(fichier)
    pos = bk.load.DLC_pos()
    pos.index = pos.index
    speeds = bk.compute.speed(pos,2, columns_to_drop=['e_tail'])
    freeze = bk.compute.freezing_intervals(speeds, 25, mode='multiple_speed', clean=True) - t_s *1_000_000
    pourcents = all_pourcentages(intervals, freeze)
    ptotal.append(pourcents)
#     bk.plot.intervals(freeze,col = 'b')
    plt.plot(np.mean([starts,ends],0), pourcents, '-o',alpha = 0.5)
plt.plot(np.mean([starts,ends],0),np.mean(ptotal,0),'*-r')
plt.legend(['Rat13', 'Rat14','Rat15','Rat16','Mean'])
plt.title('Pourcent freezing in neutral context before fear conditionning')

Y://fear_conditionning_ptsd\Rat13\Rat13-20201202\Rat13_201202_183404
Y://fear_conditionning_ptsd\Rat14\Rat14-20201203\Rat14_201203_165203
Y://fear_conditionning_ptsd\Rat15\Rat15-20201130\Rat15_201130_164002


ValueError: Length mismatch: Expected axis has 8194 elements, new values have 8195 elements

In [84]:
t_start_exp_tone = [31.5871,32.76115,5.97]

ptotal = []
plt.figure()

for fichier,t_s,d_t in zip(tone_sess,t_start_exp_tone,digitalin_tone):
    os.chdir(fichier)
    print(fichier)
    pos = bk.load.DLC_pos()
    pos.index = pos.index
    tone = bk.compute.tone_intervals(bk.load.digitalin('digitalin.dat')[d_t,:])- t_s * 1_000_000
    speeds = bk.compute.speed(pos,2, columns_to_drop=['e_tail'])
    freeze = bk.compute.freezing_intervals(speeds, 25, mode='multiple_speed', clean=True) - t_s *1_000_000
    pourcents = all_pourcentages(intervals, freeze)
    ptotal.append(pourcents)
#     bk.plot.intervals(freeze,col = 'b')
    plt.plot(np.mean([starts,ends],0), pourcents, '-o',alpha = 0.5)
    
bk.plot.intervals(tone,'orange', alpha=0.5)    
plt.plot(np.mean([starts,ends],0),np.mean(ptotal,0),'*-r')
plt.legend(['Rat13', 'Rat14', 'Rat16','Mean','Tone'])
plt.title('Pourcent freezing in tone retrieval')

Y://fear_conditionning_ptsd\Rat13\Rat13-20201203\Rat13_201203_164205
Y://fear_conditionning_ptsd\Rat14\Rat14-20201204\Rat14_201204_161859
Y://fear_conditionning_ptsd\Rat16\Rat16-20201202\Rat16_201202_182329


Text(0.5,1,'Pourcent freezing in tone retrieval')

In [82]:
t_start_exp_fear = [43.6102,61.40995,75.8984,36.58545]

ptotal = []
plt.figure()

for fichier,t_s,d_s in zip(fear_sess,t_start_exp_fear, digitalin_shock):
    os.chdir(fichier)
    pos = bk.load.DLC_pos()
    pos.index = pos.index
    tone = bk.compute.tone_intervals(bk.load.digitalin('digitalin.dat')[d_s+1,:])- t_s * 1_000_000
    shock = bk.compute.tone_intervals(bk.load.digitalin('digitalin.dat')[d_s,:])- t_s * 1_000_000
    speeds = bk.compute.speed(pos,2, columns_to_drop=['e_tail'])
    freeze = bk.compute.freezing_intervals(speeds, 25, mode='multiple_speed', clean=True) - t_s *1_000_000
    pourcents = all_pourcentages(intervals, freeze)
    ptotal.append(pourcents)
#     bk.plot.intervals(freeze,col = 'b')
    plt.plot(np.mean([starts,ends],0), pourcents, '-o',alpha = 0.5)
bk.plot.intervals(shock,'green',alpha = 0.5)    
bk.plot.intervals(tone,'orange', alpha=0.5)    
plt.plot(np.mean([starts,ends],0),np.mean(ptotal,0),'*-r')
plt.legend(['Rat13', 'Rat14', 'Rat15', 'Rat16','Mean','Shock','Tone'])
plt.title('Pourcent freezing during fear conditionning')

Text(0.5,1,'Pourcent freezing during fear conditionning')

In [83]:
t_start_exp_context = [23.49815,31.56275,48.84175,24.49015]

ptotal = []
plt.figure()

for fichier,t_s in zip(context_sess,t_start_exp_context):
    os.chdir(fichier)
    pos = bk.load.DLC_pos()
    pos.index = pos.index
    speeds = bk.compute.speed(pos,2, columns_to_drop=['e_tail'])
    freeze = bk.compute.freezing_intervals(speeds, 25, mode='multiple_speed', clean=True) - t_s *1_000_000
    pourcents = all_pourcentages(intervals, freeze)
    ptotal.append(pourcents)
#     bk.plot.intervals(freeze,col = 'b')
    plt.plot(np.mean([starts,ends],0), pourcents, '-o',alpha = 0.5)
plt.plot(np.mean([starts,ends],0),np.mean(ptotal,0),'*-r')
plt.legend(['Rat13', 'Rat14', 'Rat15', 'Rat16','Mean'])
plt.title('Pourcent freezing in context retrieval')

Text(0.5,1,'Pourcent freezing in context retrieval')

In [54]:
digit = 'Y://fear_conditionning_ptsd/Rat15/Rat15-20201130/Rat15_201130_164002/digitalin.dat'
data = bk.load.digitalin(digit)
n_TTL = bk.compute.TTL_to_times(data[0,:])
len(n_TTL)

8196

In [26]:
def CountFrames(path):
    import cv2
    i = 0
    ret = True
    cap = cv2.VideoCapture(path)
    return(int(cap.get(cv2.CAP_PROP_FRAME_COUNT)))

In [80]:
import pandas as pd
pos = pd.read_hdf('Y://fear_conditionning_ptsd/Rat15/Rat15-20201130/Rat15_201130_164002/Basler_acA1300-200uc__23039139__20201130_164007396DLC_resnet50_classical_fear_conditionningNov20shuffle1_1030000_filtered.h5')

In [81]:
pos

scorer    DLC_resnet50_classical_fear_conditionningNov20shuffle1_1030000  \
bodyparts                                                          snout   
coords                                                                 x   
0                                                 246.730301               
1                                                   0.000000               
2                                                   0.000000               
3                                                   0.000000               
4                                                   0.000000               
...                                                      ...               
8189                                              161.786927               
8190                                              163.088245               
8191                                              165.781670               
8192                                              166.487365               
8193                                              167.010364               

scorer                                                               \
bodyparts                           left_ear                          
coords              y likelihood           x           y likelihood   
0           -0.750403   0.000028  219.727676    1.191529   0.000015   
1            0.000000   0.000027    0.000000    0.000000   0.000014   
2            0.000000   0.000028    0.000000    0.000000   0.000015   
3            0.000000   0.000029    0.000000    0.000000   0.000014   
4            0.000000   0.000030    0.000000    0.000000   0.000014   
...               ...        ...         ...         ...        ...   
8189       182.910835   0.999972  152.567768  163.809196   0.998994   
8190       182.062789   0.999999  150.562997  162.680115   0.999820   
8191       182.105740   1.000000  151.768032  163.267427   0.999902   
8192       182.544974   1.000000  153.683652  164.264264   0.999905   
8193       182.811747   0.999999  154.669972  166.058067   0.999898   

scorer                                                                \
bodyparts   right_ear                             b_tail               
coords              x           y likelihood           x           y   
0          219.348541    1.119414   0.000017  252.401291   -0.361831   
1            0.000000    0.000000   0.000016    0.000000    0.000000   
2            0.000000    0.000000   0.000017    0.000000    0.000000   
3            0.000000    0.000000   0.000016    0.000000    0.000000   
4            0.000000    0.000000   0.000017    0.000000    0.000000   
...               ...         ...        ...         ...         ...   
8189       139.296551  182.095250   0.999785  102.558834  154.858998   
8190       143.782559  182.328111   0.999976  103.545874  154.675157   
8191       145.785084  181.682995   0.999975  103.137080  154.631880   
8192       145.691149  183.234128   0.999975  103.861073  154.428495   
8193       146.863473  184.034491   0.999962  103.438428  154.603658   

scorer                                                   
bodyparts                 e_tail                         
coords    likelihood           x           y likelihood  
0           0.000019  275.802368   57.950970   0.000080  
1           0.000020    0.000000    0.000000   0.000244  
2           0.000023    0.000000    0.000000   0.000414  
3           0.000026    0.000000    0.000000   0.000630  
4           0.000027    0.000000    0.000000   0.000741  
...              ...         ...         ...        ...  
8189        1.000000   83.472966  209.739414   0.999999  
8190        1.000000   83.167446  209.633745   0.999999  
8191        0.999999   83.072926  209.526120   0.999998  
8192        1.000000   82.835037  209.409723   0.999998  
8193        0.999999   83.092504  209.437027   0.999998  

[8194 rows x 15 columns]

In [53]:
v = 'Y://fear_conditionning_ptsd/Rat15/Rat15-20201130/Rat15_201130_164002/Basler_acA1300-200uc__23039139__20201130_164007396.mp4'

In [66]:
t = np.diff(n_TTL)

In [61]:
t[0]

0.03332500000000049

In [74]:
a = np.where(t.round(3) == 0.033,1,0)

In [76]:
np.sum(a == 0)

1

In [50]:
t

["'0.033325' n'est pas reconnu en tant que commande interne",
 'ou externe, un programme ex‚cutable ou un fichier de commandes.']

In [43]:
plt.hist(t,bins=250)

(array([1.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+0

In [3]:
os.chdir('Y://fear_conditionning_ptsd/Rat16/Rat16-20201202/Rat16_201202_171552')
pos = bk.load.DLC_pos()
pos.index = pos.index
speeds = bk.compute.speed(pos,2, columns_to_drop=['b_tail','e_tail'])
freeze = bk.compute.freezing_intervals(speeds, 25, mode='multiple_speed', clean=True) - t_s *1_000_000
pourcents = all_pourcentages(intervals, freeze)
ptotal.append(pourcents)
bk.plot.intervals(freeze,col = 'b')
plt.plot(np.mean([starts,ends],0), pourcents, '-o',alpha = 0.5)print(this is a unicorn branch)

UnboundLocalError: local variable 'filename' referenced before assignment